## Assignment 2 Data Analysis using Pandas

This assignment will contain 2 question with details as below. The due date is October 23 (Sunday), 2022 23:59PM. Each late day will result in 20% loss of total points.

### Question 1 (20 points) Datacamp courses

Datacamp is the online learning platform for data science courses and modules. As the only collaborator in Portugal, this course provides free access to the students in Business Analytics as the complementary resource. For this assignment, you are expected to complete the following moduels and submit your completion badges:

- [Importing & Cleaning Data with Python](https://app.datacamp.com/learn/skill-tracks/importing-cleaning-data-with-python)
- [Data Manipulation with Python](https://app.datacamp.com/learn/skill-tracks/data-manipulation-with-python)

Each badge will account for 10 points. If you are confident with the knowledge, you may skip the videos and directly finish the challenges. You can submit the badge by the end of the October if you would need more time. 

### Question 2 (80 points) Celluloid ceiling

Wonder Woman             |  Captain Marvel
:-------------------------:|:-------------------------:
![wonderwoman](https://upload.wikimedia.org/wikipedia/en/4/4f/Wonder_Woman_1984_poster.png) | ![marvel](https://upload.wikimedia.org/wikipedia/pt/5/59/Captain_Marvel_%282018%29.jpg)

Women are involved in the film industry in all roles, including as film directors, actresses, cinematographers, film producers, film critics, and other film industry professions, though women have been underrepresented in all these positions. Studies found that women have always had a presence in film acting, but have consistently been underrepresented, and on average significantly less well paid. 

In 2015, Forbes reported that "...just 21 of the 100 top-grossing films of 2014 featured a female lead or co-lead, while only 28.1% of characters in 100 top-grossing films were female... This means it’s much rarer for women to get the sort of blockbuster role which would warrant the massive backend deals many male counterparts demand (Tom Cruise in Mission: Impossible or Robert Downey Jr. in Iron Man, for example)".

Also, Forbes' analysis of US acting salaries in 2013 determined that the "...men on Forbes’ list of top-paid actors for that year made 2½ times as much money as the top-paid actresses. That means that Hollywood's best-compensated actresses made just 40 cents for every dollar that the best-compensated men made. 


In this assignment, we want to examine whether and how women representation is lacking in the film industry. We will adopt The Bechdel test as a measure of the representation of women in the film industry. The test is named after the American cartoonist Alison Bechdel in whose 1985 comic strip Dykes to Watch Out For the test first appeared. **A movie is said to meet the Bechdel test  following three criteria: (1) it has to have at least two women in it, who (2) who talk to each other, about (3) something besides a man.**

We are going to obtain the data ourselves to perform the analysis. Specifically, we will retrieve the movie metadata from IMDB (Internet Movie Database), an online database of information related to films, television programs, home videos, video games, and streaming content online – including cast, production crew and personal biographies, plot summaries, trivia, ratings, and fan and critical reviews. As of January 2020, IMDb has approximately 6.5 million titles (including episodes) and 10.4 million personalities in its database, as well as 83 million registered users.


The IMDb Top 250 is a list of the top rated 250 films, based on ratings by the registered users of the website using the methods described. We will focus on these famous movies in this analysis:

**Question 2.1** (25 points): We will retrieve the metadata of IMDb Top 250 movies from the [IMDb charts](https://www.imdb.com/chart/top/). For each movie on the list, we can scrape the following characteristics from the information page. For example, from the [page of top rated movie "The Shawshank Redemption"](https://www.imdb.com/title/tt0111161/?pf_rd_m=A2FGELUUNOQJNL&pf_rd_p=e31d89dd-322d-4646-8962-327b42fe94b1&pf_rd_r=F4QFC0SVZN1HTDHCY3C0&pf_rd_s=center-1&pf_rd_t=15506&pf_rd_i=top&ref_=chttp_tt_1), we want to extract the metadata about this movie as:
- IMDb id (0111161)
- Movie name (The Shawshank Redemption)
- Year (1994)
- Director (Frank Darabont)
- Stars (Tim Robbins, Morgan Freeman, Bob Gunton)
- Rating (9.3)
- Number of reviews (2.6M)
- Genres (Drama)
- Country (USA)
- Language (English)
- Budget (\$25,000,000)
- Gross box Office Revenue (\$28,884,504)

![imdb](https://mrfloris.com/files/images/imdb-top250-page-start.png)


After scraping the 250 movies, save the data as a dataframe ```imdb_top_movies```. 

Also, saving the dataframe to a local file ```imdb_top_movies.csv``` so that later you can load it without scraping the website twice.

Hint: You can get the links to these movies from the IMDb top chart page, and then scrape each movie page by sending the request to these links. At each movie page, the information requested are located at different sections. 

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [2]:
# Find movies of the top 250

# URL top 250 movies IMDB
url = 'http://www.imdb.com/chart/top'
page = requests.get(url)
soup = BeautifulSoup(page.text, "html.parser")

List = []
# Find all links and add to list
for link in soup.find_all('a', href=True):
    if "title/t" in link['href'] and link['href'] not in List:
        List.append(link['href'])

In [3]:
content = []

for movie in List: 
    # Set Url for each movie
    url = 'http://www.imdb.com' + movie
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    
    # Find metadata
    movieId = url.split("/tt",1)[1][:-1]
    title = soup.find('h1', {'data-testid':"hero-title-block__title"}).text
    rating = float(soup.find('span',{"class":"sc-7ab21ed2-1 jGRxWM"}).text)
    year = int(soup.find('span',{"class":"sc-8c396aa2-2 itZqyK"}).text)
    reviews = soup.find('div',{"class":"sc-7ab21ed2-3 dPVcnq"}).text
    runtime = soup.find('div',{"class":"sc-80d4314-1 fbQftq"}).find_all('li',{"class":"ipc-inline-list__item"})[2].text
    
    actorsAll = soup.find_all('div',{"class":"ipc-metadata-list-item__content-container"})[2].find_all('a')
    actors = []
    for actor in actorsAll:
        actors.append(actor.text)
    
    director = soup.find('div',{"class":"ipc-metadata-list-item__content-container"}).text
    
    genresAll = soup.find('div',{"class":"ipc-chip-list__scroller"})
    genres = []
    for genre in genresAll:
        genres.append(genre.text)
        
    origin = soup.find_all('section',{"data-testid":"Details"})[0].find_all('li', {"data-testid": "title-details-origin"})[0].find('ul').text
    detailSection = soup.find_all('section',{"data-testid":"Details"})[0].find_all('li', {"data-testid": "title-details-languages"})
    
    languages = []
    for l in detailSection[0].find_all('a'):
        languages.append(l.text)
        
    # When information is missing
    try:
        section = soup.find_all('section',{"data-testid":"BoxOffice"})
        budget = section[0].find_all('span', {'class': 'ipc-metadata-list-item__list-content-item'})[0].text[:-11]
        gross = section[0].find_all('span', {'class': 'ipc-metadata-list-item__list-content-item'})[-1].text
    except:
        budget = None
        gross = None
    
    # Merge metadata
    all_col = [movieId,title,rating,year,runtime,genres,reviews,actors,director,languages,budget,gross, origin]
    content.append(all_col)
    
cols = ["id","title","rating","year","runtime","genres","reviews","actors","director","languages","budget","gross", "origin"]
# Final dataframe
dataDF = pd.DataFrame(content,columns=cols)
dataDF

,id,title,rating,year,runtime,genres,reviews,actors,director,languages,budget,gross,origin
0,0111161,Os Condenados de Shawshank,9.3,1994,2h 22m,[Drama],2.7M,"[Tim Robbins, Morgan Freeman, Bob Gunton]",Frank Darabont,[English],"$25,000,000","$28,884,504",United States
1,0068646,O Padrinho,9.2,1972,2h 55m,"[Crime, Drama]",1.8M,"[Marlon Brando, Al Pacino, James Caan]",Francis Ford Coppola,"[English, Italian, Latin]","$6,000,000","$250,341,816",United States
2,0468569,O Cavaleiro das Trevas,9.0,2008,2h 32m,"[Action, Crime, Drama]",2.6M,"[Christian Bale, Heath Ledger, Aaron Eckhart]",Christopher Nolan,"[English, Mandarin]","$185,000,000","$1,006,234,167",United StatesUnited Kingdom
3,0071562,O Padrinho: Parte II,9.0,1974,3h 22m,"[Crime, Drama]",1.3M,"[Al Pacino, Robert De Niro, Robert Duvall]",Francis Ford Coppola,"[English, Italian, Spanish, Latin, Sicilian]","$13,000,000","$47,961,919",United States
4,0050083,Doze Homens em Fúria,9.0,1957,1h 36m,"[Crime, Drama]",783K,"[Henry Fonda, Lee J. Cobb, Martin Balsam]",Sidney Lumet,[English],"$350,000",$955,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0071411,Dersu Uzala - A Águia da Estepe,8.2,1975,2h 22m,"[Adventure, Biography, Drama]",30K,"[Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov]",Akira Kurosawa,"[Russian, Chinese]","$4,000,000","$14,480",Soviet UnionJapan
246,0083987,Gandhi,8.1,1982,3h 11m,"[Biography, Drama]",232K,"[Ben Kingsley, John Gielgud, Rohini Hattangadi]",Richard Attenborough,"[English, Hindi]","$22,000,000","$52,767,889",United KingdomIndiaUnited StatesSouth Africa
247,1454029,As Serviçais,8.1,2011,2h 26m,[Drama],458K,"[Viola Davis, Emma Stone, Octavia Spencer]",Tate Taylor,[English],"$25,000,000","$216,639,112",United StatesIndia
248,0103639,Aladdin,8.0,1992,1h 30m,"[Animation, Adventure, Comedy]",417K,"[Scott Weinger, Robin Williams, Linda Larkin]",Ron ClementsJohn Musker,[English],"$28,000,000","$504,050,219",United States


In [4]:
dataDF.to_csv("imdb_top_movies.csv")

In [5]:
# Using a copy of the original dataframe, to avoid getting lists as a string
imdb_top_movies = dataDF.copy()
imdb_top_movies

,id,title,rating,year,runtime,genres,reviews,actors,director,languages,budget,gross,origin
0,0111161,Os Condenados de Shawshank,9.3,1994,2h 22m,[Drama],2.7M,"[Tim Robbins, Morgan Freeman, Bob Gunton]",Frank Darabont,[English],"$25,000,000","$28,884,504",United States
1,0068646,O Padrinho,9.2,1972,2h 55m,"[Crime, Drama]",1.8M,"[Marlon Brando, Al Pacino, James Caan]",Francis Ford Coppola,"[English, Italian, Latin]","$6,000,000","$250,341,816",United States
2,0468569,O Cavaleiro das Trevas,9.0,2008,2h 32m,"[Action, Crime, Drama]",2.6M,"[Christian Bale, Heath Ledger, Aaron Eckhart]",Christopher Nolan,"[English, Mandarin]","$185,000,000","$1,006,234,167",United StatesUnited Kingdom
3,0071562,O Padrinho: Parte II,9.0,1974,3h 22m,"[Crime, Drama]",1.3M,"[Al Pacino, Robert De Niro, Robert Duvall]",Francis Ford Coppola,"[English, Italian, Spanish, Latin, Sicilian]","$13,000,000","$47,961,919",United States
4,0050083,Doze Homens em Fúria,9.0,1957,1h 36m,"[Crime, Drama]",783K,"[Henry Fonda, Lee J. Cobb, Martin Balsam]",Sidney Lumet,[English],"$350,000",$955,United States
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0071411,Dersu Uzala - A Águia da Estepe,8.2,1975,2h 22m,"[Adventure, Biography, Drama]",30K,"[Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov]",Akira Kurosawa,"[Russian, Chinese]","$4,000,000","$14,480",Soviet UnionJapan
246,0083987,Gandhi,8.1,1982,3h 11m,"[Biography, Drama]",232K,"[Ben Kingsley, John Gielgud, Rohini Hattangadi]",Richard Attenborough,"[English, Hindi]","$22,000,000","$52,767,889",United KingdomIndiaUnited StatesSouth Africa
247,1454029,As Serviçais,8.1,2011,2h 26m,[Drama],458K,"[Viola Davis, Emma Stone, Octavia Spencer]",Tate Taylor,[English],"$25,000,000","$216,639,112",United StatesIndia
248,0103639,Aladdin,8.0,1992,1h 30m,"[Animation, Adventure, Comedy]",417K,"[Scott Weinger, Robin Williams, Linda Larkin]",Ron ClementsJohn Musker,[English],"$28,000,000","$504,050,219",United States


**Question 2.2** (5 points) If you group the movies by release years, show the number of movies at each decade in descending order.

In [6]:
lowest_year = imdb_top_movies[["year"]].nsmallest(1, ['year']).iat[0,0]
highest_year = imdb_top_movies[["year"]].nlargest(1, ['year']).iat[0,0]
decades = []
x = 0
y = 10
# Assuming a decade starts at XXX1 and ends at XX10
while (int(np.floor(lowest_year / 10) * 10) + x) < highest_year:
    decades.append((int(np.floor(lowest_year / 10) * 10 + x), int(np.floor(lowest_year / 10) * 10) + y))
    x += 10
    y += 10
    
bins = pd.IntervalIndex.from_tuples(decades)
imdb_top_movies["decade"] = pd.cut(imdb_top_movies["year"], bins)
imdb_top_movies["decade"].value_counts()

(2000, 2010]    48
(1990, 2000]    45
(2010, 2020]    40
(1980, 1990]    24
(1970, 1980]    23
(1950, 1960]    22
(1960, 1970]    16
(1940, 1950]    12
(1930, 1940]    10
(1920, 1930]     6
(2020, 2030]     4
Name: decade, dtype: int64

**Quesion 2.3** (5 points) Show the number of movies by the distribution of runtime at quartile (0-25%, 25-50%, 50-75%, 75-100%).

In [7]:
# Add column with runtime in minutes
imdb_top_movies["runtime_minutes"] = 0
n = 0
for item in list(imdb_top_movies["runtime"]):
    if ("h" in item) and ("m" in item) and len(item) == 6:
        imdb_top_movies.iat[n, 14] = int(item[0]) * 60 + int(item[3:5])
    elif ("h" in item) and ("m" in item) and len(item) == 5:
        imdb_top_movies.iat[n, 14] = int(item[0]) * 60 + int(item[3])
    elif ("h" in item):
        imdb_top_movies.iat[n, 14] = int(item[0]) * 60
    elif ("m" in item):
        imdb_top_movies.iat[n, 14] = int(item[:-1])
    n += 1

# Find percentiles 25%, 50% and 75%
percentiles = np.quantile(list(imdb_top_movies["runtime_minutes"]), [0.25,0.5,0.75])

# Find smallest and largest runtime
lowest_runtime = imdb_top_movies[["runtime_minutes"]].nsmallest(1, ['runtime_minutes']).iat[0,0]-0.001
highest_runtime = imdb_top_movies[["runtime_minutes"]].nlargest(1, ['runtime_minutes']).iat[0,0]

quartiles = [(lowest_runtime, percentiles[0]), (percentiles[0], percentiles[1]), (percentiles[1], percentiles[2]), (percentiles[2], highest_runtime)]
bins = pd.IntervalIndex.from_tuples(quartiles)
imdb_top_movies["quartiles_runtime"] = pd.cut(imdb_top_movies["runtime_minutes"], bins)
imdb_top_movies["quartiles_runtime"].value_counts()

(44.999, 108.0]    64
(126.5, 145.0]     63
(145.0, 238.0]     62
(108.0, 126.5]     61
Name: quartiles_runtime, dtype: int64

**Question 2.4** (5 points) Show the top 10 most popular actor/actresses in terms of number of movies they have starred. 

In [8]:
list_actors = []
for a in range(len(imdb_top_movies["actors"])):
    for b in range(len(imdb_top_movies["actors"][a])):
        list_actors.append(imdb_top_movies["actors"][a][b])
        
pd.array(list_actors).value_counts().head(10)

Robert De Niro       9
Tom Hanks            6
Harrison Ford        6
Leonardo DiCaprio    6
Christian Bale       5
Charles Chaplin      5
Clint Eastwood       5
Toshirô Mifune       4
James Stewart        4
Mark Ruffalo         4
dtype: Int64

**Question 2.5** (5 points) Show the average ratings of movies across the genres and decades.

In [9]:
newDF = imdb_top_movies[["genres", "decade", "rating"]]
newDF.explode("genres").groupby(["genres", "decade"]).mean().apply(lambda x : round(x,2)).unstack("genres")

rating                                                          \
genres       Action Adventure Animation Biography Comedy Crime Drama Family   
decade                                                                        
(1920, 1930]   8.20      8.20       NaN      8.20   8.22   NaN  8.25   8.30   
(1930, 1940]    NaN      8.10       NaN       NaN   8.32  8.30  8.27   8.10   
(1940, 1950]    NaN      8.20       NaN       NaN   8.20  8.25  8.31   8.60   
(1950, 1960]   8.45      8.20       NaN       NaN   8.27  8.36  8.29    NaN   
(1960, 1970]   8.40      8.40       NaN      8.20   8.40  8.27  8.25   8.10   
(1970, 1980]   8.65      8.32       NaN      8.20   8.17  8.60  8.37    NaN   
(1980, 1990]   8.24      8.32      8.30      8.40   8.23  8.43  8.30   8.10   
(1990, 2000]   8.39      8.29      8.26      8.50   8.21  8.35  8.43   8.20   
(2000, 2010]   8.47      8.36      8.22      8.19   8.22  8.36  8.33   8.32   
(2010, 2020]   8.25      8.28      8.30      8.21   8.21  8.17  8.25   8.10   
(2020, 2030]   8.27      8.20       NaN       NaN   8.10  8.90  8.65    NaN   

                                ...                                       \
genres       Fantasy Film-Noir  ... Horror Music Musical Mystery Romance   
decade                          ...                                        
(1920, 1930]     NaN       NaN  ...    NaN   NaN     NaN     NaN    8.20   
(1930, 1940]    8.10      8.10  ...    NaN   NaN     NaN    8.20    8.32   
(1940, 1950]    8.60      8.27  ...    NaN   NaN     NaN    8.20    8.27   
(1950, 1960]    8.10       NaN  ...   8.50   8.2     8.3    8.40    8.24   
(1960, 1970]     NaN       NaN  ...    NaN   NaN     NaN    8.50     NaN   
(1970, 1980]    8.50       NaN  ...   8.33   NaN     NaN    8.35     NaN   
(1980, 1990]    8.30       NaN  ...   8.20   8.4     NaN    8.20    8.50   
(1990, 2000]    8.35       NaN  ...    NaN   NaN     NaN    8.38    8.45   
(2000, 2010]    8.15       NaN  ...    NaN   8.5     NaN    8.30    8.22   
(2010, 2020]    8.25       NaN  ...    NaN   8.5     NaN    8.13    8.10   
(2020, 2030]    8.30       NaN  ...    NaN   NaN     NaN    8.90     NaN   

                                                  
genres       Sci-Fi Sport Thriller   War Western  
decade                                            
(1920, 1930]   8.30   NaN      NaN   NaN     NaN  
(1930, 1940]    NaN   NaN     8.30  8.30     NaN  
(1940, 1950]    NaN   NaN     8.10  8.27     8.2  
(1950, 1960]    NaN   NaN     8.30  8.30     NaN  
(1960, 1970]   8.30   NaN     8.15  8.27     8.5  
(1970, 1980]   8.40  8.15     8.15  8.23     NaN  
(1980, 1990]   8.26   NaN     8.30  8.32     NaN  
(1990, 2000]   8.50  8.20     8.28  8.60     8.2  
(2000, 2010]   8.45  8.10     8.32  8.25     NaN  
(2010, 2020]   8.30  8.20     8.22  8.20     8.4  
(2020, 2030]    NaN   NaN      NaN   NaN     NaN  

[11 rows x 21 columns]

**Question 2.6** (10 points) Now let's retrieve data from Bechdel Test Movie website [for each movie](https://bechdeltest.com/). You can send the requests to the API: https://bechdeltest.com/api/v1/doc#getMovieByImdbId. For example, for the movie The Shawshank Redemption (the IMDb id: 0111161), you can simply call: http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid=0111161. 

Create a dataframe ```bechdel_imdb_top``` that merge the bechdel test info with the ```imdb_top_movies``` show how many top 250 movies are also in the bechdel test website.

In [10]:
bechdel_imdb_top = imdb_top_movies.copy()
bechdel_imdb_top["Bechdel_rating"] = float("nan")

for i in range(len(bechdel_imdb_top)):
    page = requests.get("http://bechdeltest.com/api/v1/getMovieByImdbId?imdbid="+bechdel_imdb_top["id"][i])
    soup = BeautifulSoup(page.content, 'html.parser')
    if len(str(soup))>70:
        # 8 is the number of characters the rating value is after the "r" of the word "rating"
        rating = int(str(soup)[str(soup).find("rating")+8])
        bechdel_imdb_top.iat[i, 16] = rating

bechdel_imdb_top

,id,title,rating,year,runtime,genres,reviews,actors,director,languages,budget,gross,origin,decade,runtime_minutes,quartiles_runtime,Bechdel_rating
0,0111161,Os Condenados de Shawshank,9.3,1994,2h 22m,[Drama],2.7M,"[Tim Robbins, Morgan Freeman, Bob Gunton]",Frank Darabont,[English],"$25,000,000","$28,884,504",United States,"(1990, 2000]",142,"(126.5, 145.0]",0.0
1,0068646,O Padrinho,9.2,1972,2h 55m,"[Crime, Drama]",1.8M,"[Marlon Brando, Al Pacino, James Caan]",Francis Ford Coppola,"[English, Italian, Latin]","$6,000,000","$250,341,816",United States,"(1970, 1980]",175,"(145.0, 238.0]",2.0
2,0468569,O Cavaleiro das Trevas,9.0,2008,2h 32m,"[Action, Crime, Drama]",2.6M,"[Christian Bale, Heath Ledger, Aaron Eckhart]",Christopher Nolan,"[English, Mandarin]","$185,000,000","$1,006,234,167",United StatesUnited Kingdom,"(2000, 2010]",152,"(145.0, 238.0]",3.0
3,0071562,O Padrinho: Parte II,9.0,1974,3h 22m,"[Crime, Drama]",1.3M,"[Al Pacino, Robert De Niro, Robert Duvall]",Francis Ford Coppola,"[English, Italian, Spanish, Latin, Sicilian]","$13,000,000","$47,961,919",United States,"(1970, 1980]",202,"(145.0, 238.0]",2.0
4,0050083,Doze Homens em Fúria,9.0,1957,1h 36m,"[Crime, Drama]",783K,"[Henry Fonda, Lee J. Cobb, Martin Balsam]",Sidney Lumet,[English],"$350,000",$955,United States,"(1950, 1960]",96,"(44.999, 108.0]",0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,0071411,Dersu Uzala - A Águia da Estepe,8.2,1975,2h 22m,"[Adventure, Biography, Drama]",30K,"[Maksim Munzuk, Yuriy Solomin, Mikhail Bychkov]",Akira Kurosawa,"[Russian, Chinese]","$4,000,000","$14,480",Soviet UnionJapan,"(1970, 1980]",142,"(126.5, 145.0]",0.0
246,0083987,Gandhi,8.1,1982,3h 11m,"[Biography, Drama]",232K,"[Ben Kingsley, John Gielgud, Rohini Hattangadi]",Richard Attenborough,"[English, Hindi]","$22,000,000","$52,767,889",United KingdomIndiaUnited StatesSouth Africa,"(1980, 1990]",191,"(145.0, 238.0]",2.0
247,1454029,As Serviçais,8.1,2011,2h 26m,[Drama],458K,"[Viola Davis, Emma Stone, Octavia Spencer]",Tate Taylor,[English],"$25,000,000","$216,639,112",United StatesIndia,"(2010, 2020]",146,"(145.0, 238.0]",3.0
248,0103639,Aladdin,8.0,1992,1h 30m,"[Animation, Adventure, Comedy]",417K,"[Scott Weinger, Robin Williams, Linda Larkin]",Ron ClementsJohn Musker,[English],"$28,000,000","$504,050,219",United States,"(1990, 2000]",90,"(44.999, 108.0]",0.0


In [11]:
print("There are", len(bechdel_imdb_top["Bechdel_rating"].dropna()), "movies from the top 250 of IMBD in Bechdel.")

There are 241 movies from the top 250 of IMBD in Bechdel.


**Question 2.7** (5 points) Show the percenage of movies given differen genres that has passed the test in different ways (Number from 0 to 3 (0 means no two women, 1 means no talking, 2 means talking about a man, 3 means it passes the test)) List the top 5 genres that has the highest fail rate (i.e., the movie passes 0 test)

In [12]:
rat_DF = bechdel_imdb_top[["genres", "Bechdel_rating"]]

# Get the number of movies by Bechdel rating by genres
rat_DF = rat_DF.explode("genres").groupby(["genres", "Bechdel_rating"]).size().unstack()

# Sum of movies by gender
rat_DF["sum"]=0 
for i in range(len(rat_DF)):
    rat_DF.iat[i,4] = rat_DF.iloc[i,0:4].sum()

# Transform absolute values to relative values
for i in range(len(rat_DF)):
    for n in range(4):
        rat_DF.iat[i,n] = round(rat_DF.iat[i,n] / rat_DF.iat[i,4] * 100, 2)

rat_DF = rat_DF.iloc[:,0:4]
top5 = rat_DF[0].nlargest(5)

# Top 5 genres with the highest fail rate
print("The top 5 genres with the highest fail rate is:")
display(top5)

# Complete information
print("Complete data:")
display(rat_DF)

The top 5 genres with the highest fail rate is:


genres
War          50.00
Western      33.33
Romance      30.43
Film-Noir    25.00
History      25.00
Name: 0.0, dtype: float64

Complete data:


Bechdel_rating,0.0,1.0,2.0,3.0
genres,,,,
Action,19.61,29.41,1.96,49.02
Adventure,21.67,33.33,3.33,41.67
Animation,13.04,43.48,NaN,43.48
Biography,14.81,37.04,14.81,33.33
Comedy,21.74,34.78,8.70,34.78
Crime,16.33,36.73,16.33,30.61
Drama,20.36,32.93,12.57,34.13
Family,10.00,20.00,NaN,70.00
Fantasy,7.14,28.57,14.29,50.00


**Question 2.8** (5 points) Show the top 10 highest-rated English and non-English movies that passed the test completely (rating=3), respectively. 

In [13]:
# Get movies with Bechdel rating of 3 and explode on languages 
rating_3 = bechdel_imdb_top[bechdel_imdb_top["Bechdel_rating"] == 3].explode("languages")
# Get movies that contain English in the languages column 
rating_3_English = rating_3[rating_3["languages"] == "English"].sort_values("rating", ascending=False)

print("The top 10 highest-rated English movies with Bechdel score of 3 are:")
display(rating_3_English.head(10))

# Get all movies with a language other than English
rating_3_non_English = rating_3[rating_3["languages"] != "English"].sort_values("rating", ascending=False)

rating_3_non_English_new = pd.DataFrame()
# Get all movies from dataframe above that are not in English movies dataframe
for i in range(len(rating_3_non_English)):
    if rating_3_non_English.iat[i,0] not in list(rating_3_English["id"]):
        row_to_add = bechdel_imdb_top[bechdel_imdb_top["id"] == rating_3_non_English.iat[i,0]]
        rating_3_non_English_new = pd.concat([rating_3_non_English_new, row_to_add])
        
print("The top 10 highest-rated non-English movies with Bechdel score of 3 are:")
display(rating_3_non_English_new.drop_duplicates(subset=['id']).reset_index(drop=True).head(10))

The top 10 highest-rated English movies with Bechdel score of 3 are:


,id,title,rating,year,runtime,genres,reviews,actors,director,languages,budget,gross,origin,decade,runtime_minutes,quartiles_runtime,Bechdel_rating
2,0468569,O Cavaleiro das Trevas,9.0,2008,2h 32m,"[Action, Crime, Drama]",2.6M,"[Christian Bale, Heath Ledger, Aaron Eckhart]",Christopher Nolan,English,"$185,000,000","$1,006,234,167",United StatesUnited Kingdom,"(2000, 2010]",152,"(145.0, 238.0]",3.0
5,0108052,A Lista de Schindler,9.0,1993,3h 15m,"[Biography, Drama, History]",1.3M,"[Liam Neeson, Ralph Fiennes, Ben Kingsley]",Steven Spielberg,English,"$22,000,000","$322,161,245",United States,"(1990, 2000]",195,"(145.0, 238.0]",3.0
7,0110912,Pulp Fiction,8.9,1994,2h 34m,"[Crime, Drama]",2M,"[John Travolta, Uma Thurman, Samuel L. Jackson]",Quentin Tarantino,English,"$8,000,000","$213,928,762",United States,"(1990, 2000]",154,"(145.0, 238.0]",3.0
12,1375666,A Origem,8.8,2010,2h 28m,"[Action, Adventure, Sci-Fi]",2.3M,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elli...",Christopher Nolan,English,"$160,000,000","$836,848,102",United StatesUnited Kingdom,"(2000, 2010]",148,"(145.0, 238.0]",3.0
13,0167261,O Senhor dos Anéis - As Duas Torres,8.8,2002,2h 59m,"[Action, Adventure, Drama]",1.7M,"[Elijah Wood, Ian McKellen, Viggo Mortensen]",Peter Jackson,English,"$94,000,000","$947,926,792",New ZealandUnited States,"(2000, 2010]",179,"(145.0, 238.0]",3.0
15,0133093,Matrix,8.7,1999,2h 16m,"[Action, Sci-Fi]",1.9M,"[Keanu Reeves, Laurence Fishburne, Carrie-Anne...",Lana WachowskiLilly Wachowski,English,"$63,000,000","$467,222,728",United StatesAustralia,"(1990, 2000]",136,"(126.5, 145.0]",3.0
16,0099685,Tudo Bons Rapazes,8.7,1990,2h 25m,"[Biography, Crime, Drama]",1.1M,"[Robert De Niro, Ray Liotta, Joe Pesci]",Martin Scorsese,English,"$25,000,000","$47,036,784",United States,"(1980, 1990]",145,"(126.5, 145.0]",3.0
20,0038650,Do Céu Caiu Uma Estrela,8.6,1946,2h 10m,"[Drama, Family, Fantasy]",453K,"[James Stewart, Donna Reed, Lionel Barrymore]",Frank Capra,English,"$3,180,000","$6,184,298",United States,"(1940, 1950]",130,"(126.5, 145.0]",3.0
21,0102926,O Silêncio dos Inocentes,8.6,1991,1h 58m,"[Crime, Drama, Thriller]",1.4M,"[Jodie Foster, Anthony Hopkins, Lawrence A. Bo...",Jonathan Demme,English,"$19,000,000","$272,742,922",United States,"(1990, 2000]",118,"(108.0, 126.5]",3.0
25,0816692,Interstellar,8.6,2014,2h 49m,"[Adventure, Drama, Sci-Fi]",1.8M,"[Matthew McConaughey, Anne Hathaway, Jessica C...",Christopher Nolan,English,"$165,000,000","$773,897,851",United StatesUnited KingdomCanada,"(2010, 2020]",169,"(145.0, 238.0]",3.0


The top 10 highest-rated non-English movies with Bechdel score of 3 are:


,id,title,rating,year,runtime,genres,reviews,actors,director,languages,budget,gross,origin,decade,runtime_minutes,quartiles_runtime,Bechdel_rating
0,8267604,Cafarnaum,8.4,2018,2h 6m,[Drama],89K,"[Zain Al Rafeea, Yordanos Shiferaw, Boluwatife...",Nadine Labaki,"[Arabic, Amharic]","$4,000,000","$64,417,003",LebanonFranceCyprusQatarUnited Kingdom,"(2010, 2020]",126,"(108.0, 126.5]",3.0
1,5311514,Kimi no na wa.,8.4,2016,1h 46m,"[Animation, Drama, Fantasy]",265K,"[Ryûnosuke Kamiki, Mone Kamishiraishi, Ryô Nar...",Makoto Shinkai,[Japanese],"¥370,000,000","$358,180,115",Japan,"(2010, 2020]",106,"(44.999, 108.0]",3.0
2,0119698,A Princesa Mononoke,8.4,1997,2h 14m,"[Animation, Action, Adventure]",392K,"[Yôji Matsuda, Yuriko Ishida, Yûko Tanaka]",Hayao Miyazaki,[Japanese],"¥2,400,000,000","$170,005,875",Japan,"(1990, 2000]",134,"(126.5, 145.0]",3.0
3,0048473,O Lamento da Vereda,8.3,1955,2h 5m,[Drama],33K,"[Kanu Bannerjee, Karuna Bannerjee, Subir Baner...",Satyajit Ray,[Bengali],,"$135,342",India,"(1950, 1960]",125,"(108.0, 126.5]",3.0
4,1832382,Uma Separação,8.3,2011,2h 3m,[Drama],245K,"[Payman Maadi, Leila Hatami, Sareh Bayat]",Asghar Farhadi,[Persian],"$500,000","$22,926,076",IranFranceAustralia,"(2010, 2020]",123,"(108.0, 126.5]",3.0
5,0022100,Matou,8.3,1931,1h 57m,"[Crime, Mystery, Thriller]",158K,"[Peter Lorre, Ellen Widmann, Inge Landgut]",Fritz Lang,[German],,"$35,566",Germany,"(1930, 1940]",117,"(108.0, 126.5]",3.0
6,0347149,O Castelo Andante,8.2,2004,1h 59m,"[Animation, Adventure, Family]",392K,"[Chieko Baishô, Takuya Kimura, Tatsuya Gashûin]",Hayao Miyazaki,[Japanese],"$24,000,000","$237,536,126",Japan,"(2000, 2010]",119,"(108.0, 126.5]",3.0
7,0457430,O Labirinto do Fauno,8.2,2006,1h 58m,"[Drama, Fantasy, War]",666K,"[Ivana Baquero, Ariadna Gil, Sergi López]",Guillermo del Toro,[Spanish],"$19,000,000","$83,862,032",MexicoSpain,"(2000, 2010]",118,"(108.0, 126.5]",3.0
8,4016934,A Criada,8.1,2016,2h 25m,"[Drama, Romance, Thriller]",146K,"[Kim Min-hee, Ha Jung-woo, Cho Jin-woong]",Park Chan-wook,"[Korean, Japanese]","₩10,000,000,000","$37,854,655",South Korea,"(2010, 2020]",145,"(126.5, 145.0]",3.0
9,3011894,Relatos Selvagens,8.1,2014,2h 2m,"[Comedy, Drama, Thriller]",198K,"[Darío Grandinetti, María Marull, Mónica Villa]",Damián Szifron,[Spanish],"$3,300,000","$31,061,476",ArgentinaSpainFranceUnited Kingdom,"(2010, 2020]",122,"(108.0, 126.5]",3.0


**Question 2.9** (5 points) Creat a new column ROI that measures the return of investment using (box revenue-budget)/budget, and compare the average ROI between movies that passed (rating=3) and failed the test (rating=0), are their ROI different? Explain.

In [14]:
# Remove elements blocking the convertion of value to an integer
bechdel_imdb_top["budget_str"] = bechdel_imdb_top["budget"].str.replace(',','',regex=False)
bechdel_imdb_top["gross_str"] = bechdel_imdb_top["gross"].str.replace(',','',regex=False).str.replace(' (estimated)','',regex=False)

# Consider only movies with dollar values, since we can't compare revenues/budgets in different currencies
movies_dollar = bechdel_imdb_top[bechdel_imdb_top["budget_str"].str[0] == "$"]
movies_dollar = movies_dollar.copy()
movies_dollar["budget_str"] = movies_dollar["budget_str"].str.replace('$','',regex=False)
movies_dollar["gross_str"] = movies_dollar["gross_str"].str.replace('$','',regex=False)

# Compute ROI using formula (gross rev. - budget)/budget
movies_dollar["ROI"] = 0
for i in range(len(movies_dollar)):
    movies_dollar.iat[i, 19] = (int(movies_dollar.iat[i,18]) - int(movies_dollar.iat[i,17])) / int(movies_dollar.iat[i,17])

# Dataframe with ROI values for each movie
#display(movies_dollar)

movies_dollar[["Bechdel_rating","ROI"]].groupby("Bechdel_rating").mean()

,ROI
Bechdel_rating,
0.0,5.368421
1.0,7.830769
2.0,7.047619
3.0,9.658537


#### Explanation
The average ROI for movies with a rating of 3 is higher than the average ROI for movies with a rating of 0, 9.646341 > 5.368421 . This means that movies with 2 women talking about a man and something else are financially more successful than movies where there are not 2 named women interacting.

**Question 2.10** (10 points) Now load the bech test results from all movies (https://bechdeltest.com/api/v1/doc#getAllMovies) that contains the all movies that are rated by the Bechdel Test website. Are women representation improved over the decades? Create a dataframe ```bechdel_imdb```, comparing the top 250 and other movies, in terms of percentage, how many passed/failed the test? 

In [15]:
page = requests.get("http://bechdeltest.com/api/v1/getAllMovies")
data = page.json()

In [31]:
data_DF = pd.DataFrame(data)

# Get lowest and highest year
lowest_year = data_DF[["year"]].nsmallest(1, ['year']).iat[0,0]
highest_year = data_DF[["year"]].nlargest(1, ['year']).iat[0,0]
decades = []
x = 0
y = 10
# Assuming a decade starts at XXX1 and ends at XX10
while (int(np.floor(lowest_year / 10) * 10) + x) < highest_year:
    decades.append((int(np.floor(lowest_year / 10) * 10 + x), int(np.floor(lowest_year / 10) * 10) + y))
    x += 10
    y += 10

bins = pd.IntervalIndex.from_tuples(decades)
data_DF["decade"] = pd.cut(data_DF["year"], bins)

# Create dataframes with movies in imbd and not in imbd
imbd_movies = data_DF.copy()
other_movies = data_DF.copy()
for i in range(len(other_movies)):
    if data_DF.iloc[i,3] in list(bechdel_imdb_top["id"]):
        other_movies = other_movies.drop(i)
    else:
        imbd_movies = imbd_movies.drop(i)

#display(other_movies)
#display(imbd_movies)

# Get percentages weights
other_movies_rat = other_movies[["rating"]].groupby("rating").size().apply(lambda x: round((x/len(other_movies["rating"]))*100, 2))
imbd_movies_rat = imbd_movies[["rating"]].groupby("rating").size().apply(lambda x: round((x/len(imbd_movies["rating"]))*100, 2))

display(data_DF[["decade", "rating"]].groupby("decade").mean())
print("Other movies:")
display(other_movies_rat)
print("Imbd movies:")
display(imbd_movies_rat)

,rating
decade,
"(1870, 1880]",0.000000
"(1880, 1890]",0.000000
"(1890, 1900]",0.069444
"(1900, 1910]",0.205882
"(1910, 1920]",1.692308
"(1920, 1930]",1.274725
"(1930, 1940]",2.067873
"(1940, 1950]",2.000000
"(1950, 1960]",2.006452


Other movies:


rating
0    10.91
1    21.53
2    10.10
3    57.47
dtype: float64

Imbd movies:


rating
0    20.33
1    31.12
2     9.96
3    38.59
dtype: float64